In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm

# Parsing

### Levels

In [ ]:
gauges_file = gpd.read_file('/home/anton/dima_experiments/Vip_gZ/data/initial_gauges/great_db/discharge_gauges_old/discharge_gauges.gpkg')


In [13]:
geom_file = gpd.read_file('/home/anton/dima_experiments/Vip_gZ/data/initial_gauges/great_db/discharge_gauges_old/discharge_gauges.gpkg')
geom_file = geom_file.set_index('gauge_id')
geom_file

,name_ru,name_en,area,geometry
gauge_id,,,,
49001,р.Ковда - пос.Софпорог,r.Kovda - pos.Sofporog,3520.00,POINT (31.41892 65.79876)
49014,р.Корпи-Йоки - пос.Пяозерский,r.Korpi-Joki - pos.Pjaozerskij,217.00,POINT (31.05345 65.78181)
49017,р.Тумча - пос.Алакуртти,r.Tumcha - pos.Alakurtti,2100.00,POINT (30.33082 66.95957)
49030,р.Кузема - ст.Кузема,r.Kuzema - st.Kuzema,882.00,POINT (34.23039 65.37582)
49047,р.Судно - с.Вокнаволок,r.Sudno - s.Voknavolok,1160.00,POINT (30.51036 64.95441)
...,...,...,...,...
72477,р.Кунья - д.Уварово,r.Kun'ja - d.Uvarovo,2480.00,POINT (30.93000 56.78000)
72641,руч.Мельничный - д.Брод,ruch.Mel'nichnyj - d.Brod,4.24,POINT (28.40000 58.63000)
72680,кан.Магистральный - д.Люцково г/ств.N1,kan.Magistral'nyj - d.Ljutskovo g/stv.N1,5.21,POINT (28.51000 56.73000)


In [10]:
heigh_bs = pd.read_csv('./data/levels/height_id.csv')
heigh_bs = heigh_bs.rename(columns={'Unnamed: 0': 'gauge_id'})
heigh_bs['gauge_id'] = list(map(str, heigh_bs['gauge_id']))
heigh_bs = heigh_bs.set_index('gauge_id')
heigh_bs = heigh_bs.drop_duplicates()
heigh_bs

,name,height
gauge_id,,
75389,ПРОТВА - P.ПPOTBA - C.CПAC-ЗAГOPЬE,119.04
75394,ОСЕТР - P.OCETP - Д.MAPKИHO,107.00
75395,МОСКВА - P.MOCKBA - Д.БAPCУKИ,183.00
75402,МОСКВА - P.MOCKBA - Г.ЗBEHИГOPOД,132.89
75403,МОСКВА - P.MOCKBA - C.ПETPOВО-ДAЛЬHEE,127.50
...,...,...
49048,"Войница (Каарто-йоки, Терва-йоки) - P.BOЙHИЦA ...",101.17
49052,"Ухта (Ухтуа-йоки, Ухтуска) - P.УXTA - ПГT KAЛE...",114.32
49053,Р.НОРВИ-ЙОКИ - P.HOPBИ-ЙOKИ - ПГT KAЛEBAЛA,104.00


In [11]:
level_storage = './data/levels/final'
discharge_storage = './data/discharge/final'

In [23]:
dis_file = pd.read_csv(f'{discharge_storage}/{gauge}.csv',
                    index_col='date')
dis_file.index = pd.to_datetime(dis_file.index)
ws_area = geom_file.loc[gauge, 'new_area']
dis_file['q_mm_day'] = (dis_file['discharge'] * 86400)/(ws_area * 1e3)
dis_file = dis_file.rename(columns={'discharge': 'q_cms_s'})
dis_file = dis_file[~dis_file.index.duplicated(keep='first')]
dis_file

,q_cms_s,q_mm_day
date,,
2008-01-01,NaN,NaN
2008-01-02,NaN,NaN
2008-01-03,NaN,NaN
2008-01-04,NaN,NaN
2008-01-05,NaN,NaN
...,...,...
2020-12-27,1080.0,0.508497
2020-12-28,1080.0,0.508497
2020-12-29,1080.0,0.508497


In [15]:
hydro_storage = '/home/anton/dima_experiments/my_dissertation/geo_data/great_db/hydro_meteo'

In [22]:
for gauge in geom_file.index:
    try:
        lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                               index_col='date')
        lvl_file.index = pd.to_datetime(lvl_file.index)
        
        try:
            lvl_bs = heigh_bs.loc[gauge, 'height']
        except KeyError:
            lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                                        index_col='date')
            lvl_file.index = pd.to_datetime(lvl_file.index)
            lvl_bs = np.NaN
            lvl_file['lvl_mbs'] = lvl_bs + \
                lvl_file['level'] * 1e-2  # type: ignore
            lvl_file = lvl_file.rename(columns={'level': 'lvl_sm'})
            lvl_file = lvl_file[~lvl_file.index.duplicated(keep='first')]
            lvl_file.to_csv(
                f'{hydro_storage}/h/{gauge}.csv')

        lvl_file['lvl_mbs'] = lvl_bs + \
            lvl_file['level'] * 1e-2  # type: ignore
        lvl_file = lvl_file.rename(columns={'level': 'lvl_sm'})
        lvl_file = lvl_file[~lvl_file.index.duplicated(keep='first')]
        lvl_file.to_csv(
            f'{hydro_storage}/h/{gauge}.csv')
    except FileNotFoundError:
        lvl_file = pd.DataFrame()
        lvl_file['date'] = pd.date_range(start='01/01/2008',
                                         end='12/31/2020')
        lvl_file = lvl_file.set_index('date')
        lvl_file['lvl_sm'] = np.NaN
        lvl_file['lvl_mbs'] = np.NaN

    try:
        dis_file = pd.read_csv(f'{discharge_storage}/{gauge}.csv',
                               index_col='date')
        dis_file.index = pd.to_datetime(dis_file.index)
        try:
            ws_area = geom_file.loc[gauge, 'new_area']
        except KeyError:
            ws_area = np.NaN
        dis_file['q_mm_day'] = (dis_file['discharge'] * 86400)/(ws_area * 1e3)
        dis_file = dis_file.rename(columns={'discharge': 'q_cms_s'})
        dis_file = dis_file[~dis_file.index.duplicated(keep='first')]
        dis_file.to_csv(
            f'{hydro_storage}/q/{gauge}.csv')
    except FileNotFoundError:
        dis_file = pd.DataFrame()
        dis_file.index = pd.date_range(start='01/01/2008',
                                       end='12/31/2020')
        dis_file['date'] = pd.date_range(start='01/01/2008',
                                         end='12/31/2020')
        dis_file = dis_file.set_index('date')
        dis_file['q_cms_s'] = np.NaN
        dis_file['q_mm_day'] = np.NaN

    res_file = pd.concat([dis_file, lvl_file], axis=1)
    res_file = res_file[['lvl_sm', 'q_cms_s', 'lvl_mbs', 'q_mm_day']]
    res_file.to_csv(
        f'{hydro_storage}/q_h/{gauge}.csv')
